## **Load libraries and data**

In [ ]:
!pip install --upgrade pythainlp
!pip install pyLDAvis==2.1.2
!pip install emoji

     |████████████████████████████████| 11.0MB 16.6MB/s 
     |████████████████████████████████| 747kB 44.4MB/s 
     |████████████████████████████████| 1.6MB 14.5MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=f326654cf6531e00bfbc7004b47f07178ee1f01994515d2cc7e9476ed6cf0672
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis
     |████████████████████████████████| 133kB 15.3MB/s 


In [ ]:
import pandas as pd
import pythainlp
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from pythainlp.util import dict_trie
from pythainlp.corpus.common import thai_words
import string
import re
import emoji
from pythainlp.tag import pos_tag

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/CRM/3 Voice of Customer Analytics/CustomerReviews.csv')

In [ ]:
df.tail()

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating
16,17,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,ployynp,บุฟเฟ่ต์ชาบูและพิซซ่าไม่อั้นในราคา 199 บาท เน้...,หลังจากที่เคยลองสาขายูเนี่ยนมอลล์ไป รอบนี้มาที...,4.0
17,18,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,27a91236fe5e4559a4f097c97a480781,ร้านบุฟเฟ่ต์ ราคามิตรภาพ อยู่ชั้น4 ติดโรงหนัง ...,ร้านบุฟเฟ่ต์ที่มีโปรโมชั่นหัวละ199บาท ไม่รวมน้...,4.0
18,19,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,0b81d251e4db486f9bcdba73b374ed99,ของหลากหลาย ปนๆ งงๆ นิดหน่อย,เคยรู้จักร้านนี้จากที่ union mall ไม่เคยได้ลอง...,3.0
19,20,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,40e0e087f3914fd49a8933b5a29936ca,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,5.0
20,21,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,41841cb99ea243a3a8d4b006e946c586,แม้จะแปลกบ้าง แต่ก็ถือว่าอยู่ในเกณฑ์ที่ดี มีอา...,ก็ตามที่เขียนเลยครับ ว่า ถ้าจะจ่ายในราคา 199 บ...,NaN


In [ ]:
df['Review'] = df['Review'].replace('\n',' ')

## **Tokenize Words**

In [ ]:
new_words = {'น้ำซุป','สันคอ','คุณ​ภาพ','โมโม่','โมโม','มาตราฐาน','Terminal 21','ซอยซอส','คุโรบูตะ','สตอเบอรี่','แย้ววว','บุฟเฟ่',
             'พระราม 9','ไก่บด','มากก','ไก่ไม้ไผ่','ถั่วแดง','ออส','หอม','พอนซึ','พาราไดส์','เดอะมอล','บางกะปิ','ประยุคก์','โชยุ',
             'ขาบูชิ','เทมปุระ','ชาบูชิ','ญี่ปุน','เบค่อน','เดอะมอลล์','โอเด้ง','หมุ','วนิลา','สตอเบอร์รี่','ชอคโกแลต','อารามณ์','ชูชิ',
             'บุฟเฟ','เก้บ','นํ้าจิ้ม','แมคโดนัล','ยูเนี่ยนมอลล์','นารายพิซเซอเรีย','รีฟีล','น้ำซึป','ซีฟู้ด','ฮันนี่โทส','Narai pizzeria',
             'สปาเกตตี้','นิวออลีน','บุพเฟ่','รสชาต','บราวนี่','วิปครีม'}
  
words = new_words.union(thai_words())

custom_dictionary_trie = dict_trie(words)
#word_tokenize(text, custom_dict=custom_dictionary_trie)

In [ ]:
def remove_emoji(text):
    return emoji.get_emoji_regexp().sub(u'', text)

In [ ]:
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = [' ', '  ', '\n', 'ร้าน', '(', ')','\u200b',',','–','/','ชาบู',
                 'กิน','ทาน','อาหาร','บาท','สั่ง']
screening_words = stopwords + removed_words
table = str.maketrans('', '', string.punctuation)


def tokenize_with_space(sentence):
  merged = ''
  #words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  # sentence = sentence.replace(':','')
  sentence = sentence.translate(table)
  sentence = remove_emoji(sentence)
  sentence = sentence.replace('เเ','แ')
  sentence = sentence.replace('\u200b','')
  words = pythainlp.word_tokenize(str(sentence), custom_dict=custom_dictionary_trie, engine='newmm')
  words_pos = pos_tag(words,corpus='orchid_ud')
  for word in words_pos:
    if word[0] not in screening_words:
      # print(word)
      if word[1] in ['NOUN']: #,'VERB','ADJ','ADV'
        if not word[0].isnumeric():
          merged = merged + ',' + word[0]
  return merged[1:]

In [ ]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [ ]:
df[['Review','Review_tokenized']].head()

,Review,Review_tokenized
0,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,"พนักงาน,น่ารัก,สะอาดสะอ้าน,รสชาติ,ความต้องการ,..."
1,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,"โมโม่,พาราไดซ์,คุณภาพ,เค้า,มาตราฐาน,มีโอ,กา,สม..."
2,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,"โต๊ะ,หม้อ,โต๊ะ,ฉาก,กั้น,รส,น้ำจิ้ม,น้ำซุป,น้ำ,..."
3,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,"เนื้อ,ราคา,โหด,โมโม่,โปร,ส่วนตัว,เนื้อ,ออส,ไว,..."
4,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,"หอ,โปร,บัตรเครดิต,น้ำ,เนื้อ,ต่าง ๆ,ผัก,เส้น,สา..."


In [ ]:
df['Restaurant'].unique()

array(['Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ',
       'Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G', 'ข้าน้อยขอชาบู'],
      dtype=object)

# **MoMo**

## Create Dictionary

In [ ]:
df_momo = df[df['Restaurant'] == 'Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ']
df_momo.shape

(8, 8)

In [ ]:
documents_momo = df_momo['Review_tokenized'].to_list()
texts_momo = [[text for text in doc.split(',')] for doc in documents_momo]
dictionary_momo = gensim.corpora.Dictionary(texts_momo)

In [ ]:
documents_momo

['พนักงาน,น่ารัก,สะอาดสะอ้าน,รสชาติ,ความต้องการ,เนื้อ,สันคอ,ถาด,เป็ด,แฟน,เนื้อ,คุณภาพดี,ดำ,เผ็ด,น้ำจิ้ม,อร่อย,รสชาติ,รสชาติ,ราคา,คุ้ม,โมจิ,อร่อย,ข้างนอก,ประทับใจ,บริการ,คุณภาพ',
 'โมโม่,พาราไดซ์,คุณภาพ,เค้า,มาตราฐาน,มีโอ,กา,สมาทาน,นาที,ที่ทาง,แจ้ง,รายละเอียด,การงด,ออร์เดอร์,           ,สาขา,พนักงานบริการ,น่ารัก,เหวี่ยง,สาขา,พระราม 9,พนักงาน,CTW,พรีเมียม,สาขา,Terminal 21,ลูกค้า',
 'โต๊ะ,หม้อ,โต๊ะ,ฉาก,กั้น,รส,น้ำจิ้ม,น้ำซุป,น้ำ,น้ำซุป,เห็ด,น้ำ,หวาน,เค็ม,รสชาติ,เสิร์ฟ,ไข่,สด,เป็นผู้ใหญ่,หวาน,น้ำซุป,เห็ด,ภาษา,เสิร์ฟ,เห็ด,น้ำจิ้ม,ซอยซอส,รสชาติ,จืด,แนว,น้ำซุป,เนื้อสัตว์,เนื้อวัว,เนื้อหมู,เนื้อ,เป็ด,ไก่ไม้ไผ่,เนื้อวัว,อร่อย,ชิ้น,เนื้อ,นุ่ม,น้ำ,เนื้อหมู,คุโรบูตะ,อร่อย,นุ่ม,ลิ้น,เนื้อ,เป็ด,ไก่ไม้ไผ่,ไก่บด,ลืม,รสชาติ,กรุบ,แตา,โควิด,บาร์,ผัก,น้ำ,พนักงาน,ของหวาน,ไอติม,ไดฟูกุ,ลอง,ไดฟูกุ,ไอติม,แย้ววว,สตอเบอรี่,ชาเขียว,ถั่วแดง,อร่อย,ราคา,คนละ',
 'เนื้อ,ราคา,โหด,โมโม่,โปร,ส่วนตัว,เนื้อ,ออส,ไว,ไอศกรีม,รส,ชาเขียว,รส,ข้าว,ถ้วย',
 'หอ,โปร,บัตรเครดิต,น้ำ,เนื้อ,ต่าง ๆ,ผัก,เส้น,สาขา,สาขา,CTW,เด็ก,นั่ง,สบายดี

In [ ]:
print(dictionary_momo.token2id.keys())

dict_keys(['ข้างนอก', 'ความต้องการ', 'คุณภาพ', 'คุณภาพดี', 'คุ้ม', 'ดำ', 'ถาด', 'น่ารัก', 'น้ำจิ้ม', 'บริการ', 'ประทับใจ', 'พนักงาน', 'รสชาติ', 'ราคา', 'สะอาดสะอ้าน', 'สันคอ', 'อร่อย', 'เนื้อ', 'เป็ด', 'เผ็ด', 'แฟน', 'โมจิ', '           ', 'CTW', 'Terminal 21', 'กา', 'การงด', 'ที่ทาง', 'นาที', 'พนักงานบริการ', 'พระราม 9', 'พรีเมียม', 'พาราไดซ์', 'มาตราฐาน', 'มีโอ', 'รายละเอียด', 'ลูกค้า', 'สมาทาน', 'สาขา', 'ออร์เดอร์', 'เค้า', 'เหวี่ยง', 'แจ้ง', 'โมโม่', 'กรุบ', 'กั้น', 'ของหวาน', 'คนละ', 'คุโรบูตะ', 'จืด', 'ฉาก', 'ชาเขียว', 'ชิ้น', 'ซอยซอส', 'ถั่วแดง', 'นุ่ม', 'น้ำ', 'น้ำซุป', 'บาร์', 'ผัก', 'ภาษา', 'รส', 'ลอง', 'ลิ้น', 'ลืม', 'สด', 'สตอเบอรี่', 'หม้อ', 'หวาน', 'เค็ม', 'เนื้อวัว', 'เนื้อสัตว์', 'เนื้อหมู', 'เป็นผู้ใหญ่', 'เสิร์ฟ', 'เห็ด', 'แตา', 'แนว', 'แย้ววว', 'โควิด', 'โต๊ะ', 'ไก่บด', 'ไก่ไม้ไผ่', 'ไข่', 'ไดฟูกุ', 'ไอติม', 'ข้าว', 'ถ้วย', 'ส่วนตัว', 'ออส', 'โปร', 'โหด', 'ไว', 'ไอศกรีม', 'ขนม', 'ต่าง ๆ', 'นั่ง', 'บัตรเครดิต', 'สบายดี', 'หอ', 'เด็ก', 'เส้น', 'คามี', 'บ๋วย', 'พอนซึ', 

In [ ]:
gensim_corpus_momo = [dictionary_momo.doc2bow(text, allow_update=True) for 
                      text in texts_momo]
word_frequencies_momo = [[(dictionary_momo[id], frequence) for 
                          id, frequence in couple] for 
                         couple in gensim_corpus_momo]

## Topic Modeling

In [ ]:
num_topics = 3
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary_momo[0]  # This is only to "load" the dictionary.
id2word_momo = dictionary_momo.id2token

%time model_momo = gensim.models.LdaModel(corpus=gensim_corpus_momo, \
                                          id2word=id2word_momo, \
                                          chunksize=chunksize, \
                                          alpha='auto', eta='auto', \
                                          iterations=iterations, \
                                          num_topics=num_topics, \
                                          passes=passes, eval_every=eval_every)

CPU times: user 107 ms, sys: 1.67 ms, total: 109 ms
Wall time: 112 ms


In [ ]:
pyLDAvis.gensim.prepare(model_momo, gensim_corpus_momo, dictionary_momo)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.032127 -0.003277       1        1  45.143468
2      0.019601 -0.020678       2        1  35.439442
1      0.012526  0.023955       3        1  19.417090, topic_info=          Term       Freq      Total Category  logprob  loglift
38        สาขา   4.000000   4.000000  Default  30.0000  30.0000
95      ต่าง ๆ   1.000000   1.000000  Default  29.0000  29.0000
23         CTW   1.000000   1.000000  Default  28.0000  28.0000
17       เนื้อ  11.000000  11.000000  Default  27.0000  27.0000
35  รายละเอียด   2.000000   2.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
13        ราคา   0.681723   4.830104   Topic3  -4.2122  -0.3190
18        เป็ด   0.452436   3.130788   Topic3  -4.6222  -0.2954
68        หวาน   0.446915   3.107194   Topic3  -4.6344  -0.3001
51     ชาเขียว   0.445525   3.101291   Topic3  -4.6375  -0.3013
74      เสิร์ฟ   0.444777   3.097986   Topic3  -4.6392  -0.3019

[196 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
23        3  0.663899       CTW
110       2  0.943404      Menu
111       2  0.943579    Number
112       2  0.946511  กรรมวิธี
44        1  0.859620      กรุบ
...     ...       ...       ...
92        1  0.914800        ไว
85        1  0.751068     ไอติม
85        2  0.375534     ไอติม
93        1  0.915461   ไอศกรีม
141       2  0.922243   ไอศครีม

[147 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2])

In [ ]:
model_momo.show_topics()

[(0,
  '0.051*"เนื้อ" + 0.036*"น้ำ" + 0.032*"อร่อย" + 0.028*"น้ำซุป" + 0.028*"รสชาติ" + 0.027*"น้ำจิ้ม" + 0.019*"ราคา" + 0.019*"พนักงาน" + 0.019*"ผัก" + 0.017*"เห็ด"'),
 (1,
  '0.046*"เนื้อ" + 0.040*"สาขา" + 0.032*"น้ำ" + 0.027*"อร่อย" + 0.022*"รสชาติ" + 0.020*"น้ำจิ้ม" + 0.019*"พนักงาน" + 0.019*"ผัก" + 0.017*"น้ำซุป" + 0.015*"ต่าง ๆ"'),
 (2,
  '0.047*"เนื้อ" + 0.040*"น้ำ" + 0.039*"รสชาติ" + 0.033*"น้ำจิ้ม" + 0.027*"อร่อย" + 0.025*"ราคา" + 0.023*"น้ำซุป" + 0.017*"สาขา" + 0.016*"พนักงาน" + 0.016*"ผัก"')]

In [ ]:
model_momo.show_topic(1)

[('เนื้อ', 0.04625428),
 ('สาขา', 0.040242866),
 ('น้ำ', 0.03212722),
 ('อร่อย', 0.027033392),
 ('รสชาติ', 0.021717483),
 ('น้ำจิ้ม', 0.019536817),
 ('พนักงาน', 0.019203274),
 ('ผัก', 0.019175302),
 ('น้ำซุป', 0.016831892),
 ('ต่าง ๆ', 0.015405268)]

In [ ]:
df_momo['topics'] = df_momo['Review_tokenized'].apply(lambda x: model_momo.get_document_topics(dictionary_momo.doc2bow(x.split(',')))[0][0])
df_momo['score'] = df_momo['Review_tokenized'].apply(lambda x: model_momo.get_document_topics(dictionary_momo.doc2bow(x.split(',')))[0][1])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df_momo.head(10)

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating,Review_tokenized,topics,score
0,1,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,7b16469831074f7abc7824745ee75212,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,5.0,"พนักงาน,น่ารัก,สะอาดสะอ้าน,รสชาติ,ความต้องการ,...",1,0.997714
1,2,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,pakkaramonpondej,รสชาติเหมือนทุกสาขา แต่สาขานี้ บริการดี ที่นั่งดี,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,5.0,"โมโม่,พาราไดซ์,คุณภาพ,เค้า,มาตราฐาน,มีโอ,กา,สม...",0,0.996981
2,3,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,saanowy,ชาบูพรีเมี่ยมสไตล์ญี่ปุ่น เนื้อดีมากกก,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,4.0,"โต๊ะ,หม้อ,โต๊ะ,ฉาก,กั้น,รส,น้ำจิ้ม,น้ำซุป,น้ำ,...",2,0.998904
3,4,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ployynp,เนื้อดี ไอติมดี คุ้มค่าดี,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,4.0,"เนื้อ,ราคา,โหด,โมโม่,โปร,ส่วนตัว,เนื้อ,ออส,ไว,...",1,0.996050
4,5,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,665a902b335b434ab489f00f2fbb477e,อาหารมีคุณภาพ บริการดีค่ะ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,5.0,"หอ,โปร,บัตรเครดิต,น้ำ,เนื้อ,ต่าง ๆ,ผัก,เส้น,สา...",1,0.996877
5,6,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,12f2b56b28bb4849a92690cedc1cc50a,กินให้อิ่ม ที่ momo,ร้านบุฟเฟ่ ชาบูแนวญี่ปุ่น สายเนื้อหมู เนื้อวัว...,4.0,"แนว,สาย,เนื้อหมู,เนื้อวัว,โซน,ผัก,หยิบ,เนื้อ,พ...",0,0.995480
6,7,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,f2628bf47a394345b133fec76a79661d,ชาบู - สุกกี้สไลต์ญี่ปุ่นที่แท้ทรู,Number 20 : โมโม – พาราไดส์ (สาขาเดอะมอลบางกะป...,4.0,"Number,โมโม,พาราไดส์,สาขา,เดอะมอล,บางกะปิ,ร้าน...",1,0.998300
7,8,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,backscrash,ร้านชาบูเฟรนไชส์ ที่ราคาเหมาะสมกับคุณภาพ ถือว่...,ร้านชาบูเฟรนไชส์รสชาติดีมากคุ้มค่าเหมาะสมกับรา...,4.0,"รสชาติ,ราคา,เรื่อง,น้ำซุป,งวด,น้ำ,ชาเขียว,กาแฟ...",2,0.996485


# **Shabushi**

## Create Dictionary

In [ ]:
df_shabushi = df[df['Restaurant'] == 'Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G']
df_shabushi.shape

(8, 8)

In [ ]:
documents_shabushi = df_shabushi['Review_tokenized'].to_list()
texts_shabushi = [[text for text in doc.split(',')] for doc in documents_shabushi]
dictionary_shabushi = gensim.corpora.Dictionary(texts_shabushi)

In [ ]:
print(dictionary_shabushi.token2id.keys())

dict_keys(['กุ้ง', 'ของกิน', 'คุ้ม', 'ซูชิ', 'ซ่า', 'ตอน', 'ปูอัด', 'ราคา', 'สายพาน', 'หิว', 'ฮ่า', 'าน', 'เกี๊ยว', 'เทมปุระ', 'เมนู', 'แพง', 'ไม่งั้น', 'กลาย', 'ชาบูชิ', 'ชื่นใจ', 'ญี่ปุน', 'ตัว', 'ทอด', 'ปลา', 'ผิดหวัง', 'ลืม', 'อาหารเสริม', 'เป็นมา', 'เป็นหลัก', 'เย็น', 'ไข่', 'net', 'ของคาว', 'ชม', 'น้ำซุป', 'บุฟเฟ่ต์', 'รส', 'ลายตา', 'สด', 'สไลด์', 'อัพเดท', 'อิ่ม', 'เนื้อ', 'เบค่อน', 'เพลิน', 'ไลน์', 'G', 'Net', 'Shabushi', 'กรอ', 'ของหวาน', 'ขาดสารอาหาร', 'ขึ้นราคา', 'ข่าว', 'คน', 'คนเดียว', 'คิว', 'คุณภาพ', 'งาน', 'จาน', 'ชั่วโมง', 'ชั้น', 'ชื่น', 'ตลับ', 'ตับ', 'ถูกใจ', 'นาที', 'นุ่ม', 'น้ำ', 'น้ำมัน', 'บางกะปิ', 'ปลาดิบ', 'ปอกเปลือก', 'ฝรั่ง', 'พนักงาน', 'มื้อ', 'รสชาติ', 'ระดับ', 'รายการ', 'ลูก', 'ลูกชิ้น', 'วันหยุด', 'วานิลา', 'สะอาด', 'สาขา', 'สาเหตุ', 'สไปรท์', 'ส่วนตัว', 'หมู', 'หวาน', 'หอม', 'หอย', 'หาไม่', 'ห้าง', 'อย่า', 'อร่อย', 'อาทิตย์', 'เดอะมอลล์', 'เด้ง', 'เบคอน', 'เปรี้ยวหวาน', 'เรื่อง', 'เวลา', 'เสาร์', 'แก้ว', 'แถม', 'แป้ง', 'แย่', 'โซน', 'โดน', 'โดยประมาณ', 

In [ ]:
gensim_corpus_shabushi = [dictionary_shabushi.doc2bow(text, allow_update=True) for 
                          text in texts_shabushi]
word_frequencies_shabushi = [[(dictionary_shabushi[id], frequence) for 
                              id, frequence in couple] for 
                             couple in gensim_corpus_shabushi]

## Topic Modeling

In [ ]:
num_topics = 3
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary_shabushi[0]  # This is only to "load" the dictionary.
id2word_shabushi = dictionary_shabushi.id2token

%time model_shabushi = gensim.models.LdaModel(corpus=gensim_corpus_shabushi, 
                                              id2word=id2word_shabushi, 
                                              chunksize=chunksize, \
                                              alpha='auto', eta='auto', \
                                              iterations=iterations, 
                                              num_topics=num_topics, \
                                              passes=passes, 
                                              eval_every=eval_every)

CPU times: user 143 ms, sys: 542 µs, total: 144 ms
Wall time: 146 ms


In [ ]:
pyLDAvis.gensim.prepare(model_shabushi, gensim_corpus_shabushi, dictionary_shabushi)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.025683 -0.002227       1        1  58.010037
2      0.025132 -0.002376       2        1  33.322128
1      0.000551  0.004603       3        1   8.667835, topic_info=         Term       Freq      Total Category  logprob  loglift
54         คน   9.000000   9.000000  Default  30.0000  30.0000
7        ราคา   5.000000   5.000000  Default  29.0000  29.0000
76     รสชาติ   3.000000   3.000000  Default  28.0000  28.0000
3        ซูชิ   5.000000   5.000000  Default  27.0000  27.0000
0        กุ้ง  11.000000  11.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
56        คิว   0.335071   4.824084   Topic3  -4.3909  -0.2215
97  เดอะมอลล์   0.239229   3.149833   Topic3  -4.7278  -0.1321
70    บางกะปิ   0.238627   3.145037   Topic3  -4.7303  -0.1331
22        ทอด   0.238368   3.142955   Topic3  -4.7314  -0.1336
21        ตัว   0.208930   3.487303   Topic3  -4.8632  -0.3693

[208 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
46        1  0.690019         G
46        2  0.345010         G
48        1  0.516534  Shabushi
31        2  0.976239       net
150       2  0.981628   กลยุทธ์
...     ...       ...       ...
112       1  0.513197      โอเค
30        1  0.903187       ไข่
45        2  0.976325      ไลน์
118       1  0.502972   ไอศกรีม
118       2  0.502972   ไอศกรีม

[150 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2])

In [ ]:
model_shabushi.show_topics()

[(0,
  '0.044*"กุ้ง" + 0.038*"คน" + 0.028*"สด" + 0.023*"เทมปุระ" + 0.021*"เวลา" + 0.020*"เนื้อ" + 0.018*"สาขา" + 0.018*"คิว" + 0.016*"ราคา" + 0.016*"ห้าง"'),
 (1,
  '0.029*"คน" + 0.026*"กุ้ง" + 0.022*"ราคา" + 0.019*"ซูชิ" + 0.019*"รสชาติ" + 0.018*"เวลา" + 0.017*"สด" + 0.017*"เทมปุระ" + 0.017*"สไลด์" + 0.016*"สายพาน"'),
 (2,
  '0.026*"กุ้ง" + 0.024*"สายพาน" + 0.023*"ราคา" + 0.022*"คน" + 0.020*"เมนู" + 0.019*"เทมปุระ" + 0.018*"เนื้อ" + 0.018*"ซูชิ" + 0.017*"คุ้ม" + 0.016*"ห้าง"')]

In [ ]:
model_shabushi.show_topic(1)

[('คน', 0.028822595),
 ('กุ้ง', 0.026461031),
 ('ราคา', 0.02177378),
 ('ซูชิ', 0.019440416),
 ('รสชาติ', 0.019255359),
 ('เวลา', 0.017980732),
 ('สด', 0.01721154),
 ('เทมปุระ', 0.01712978),
 ('สไลด์', 0.01672055),
 ('สายพาน', 0.015632581)]

In [ ]:
df_shabushi['topics'] = df_shabushi['Review_tokenized'].apply(lambda x: model_shabushi.get_document_topics(dictionary_shabushi.doc2bow(x.split(',')))[0][0])
df_shabushi['score'] = df_shabushi['Review_tokenized'].apply(lambda x: model_shabushi.get_document_topics(dictionary_shabushi.doc2bow(x.split(',')))[0][1])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df_shabushi.head(10)

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating,Review_tokenized,topics,score
8,9,4621tm-shabushi-เดอะมอลล์บางกะปิ-ชั้น-g,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,a477a5812ada4d5b80fd04fca22bdbca,อร่อยคุ้ม ของทานเล่นเยอะมาก,มา านที่ขาบูชิต้องมาตอนหิว ไม่งั้นจะไม่คุ้มนะค...,4.0,"าน,ตอน,หิว,ไม่งั้น,ฮ่า,เมนู,ของกิน,สายพาน,ซูชิ...",2,0.996573
9,10,4621tm-shabushi-เดอะมอลล์บางกะปิ-ชั้น-g,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,fd2ba94ec1bc4948ae109e2cb6c9cbc6,ทานอีก ชอบกุ้งทอดเทมปุระ,ใครชอบกุ้งทอดเทมปุระ แค่กุ้งเทมปุระก็คุ้มแล้ว ...,3.0,"กุ้ง,ทอด,เทมปุระ,กุ้ง,เทมปุระ,ผิดหวัง,กุ้ง,เทม...",0,0.997147
10,11,4621tm-shabushi-เดอะมอลล์บางกะปิ-ชั้น-g,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,ployynp,บุฟเฟ่ต์คุ้มๆในห้าง,กลับมาอัพเดทราคาชาบูชิ ตอนนี้อยู่ที่ 399 บาท n...,3.0,"อัพเดท,ราคา,net,ชม,น้ำซุป,รส,ซูชิ,ลายตา,เทมปุร...",2,0.997223
11,12,4621tm-shabushi-เดอะมอลล์บางกะปิ-ชั้น-g,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,c08b2776067a4026b882abfc6fca8330,เปลี่ยนจากกุ้งขาดสารอาหารเป็นกุ้งปอกเปลือกแล้ว...,ห่างหายไปนานสำหรับชาบูชิ ตั้งแต่ทางร้านได้ปรับ...,3.0,"ชาบูชิ,ขึ้นราคา,แถม,เวลา,คุณภาพ,แย่,ได้ยิน,ข่า...",0,0.999459
12,13,4621tm-shabushi-เดอะมอลล์บางกะปิ-ชั้น-g,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,a96ec47b6cff4bd1834c90d603ef0a7e,อิ่ม อร่อย ฝุดฝุด,เมื่อหลายวันก่อนนัดหาข้าวทานกับเดอะแกงค์ และก็...,4.0,"วันก่อน,ข้าว,เดอะ,แกงค์,บุฟเฟ่ต์,อยุ่,ชั้น,G,ร...",1,0.996704
13,14,4621tm-shabushi-เดอะมอลล์บางกะปิ-ชั้น-g,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,a6991f1e0e9b4916ab3ca91d86f8e481,จัดให้เต็มคราบกับชั่วโมงครึ่ง@Shabushi,บอกตรงๆว่าหลายครั้งที่เลือกจะกินบุฟเฟต์จะต้องอ...,3.0,"บุฟเฟต์,คิว,Shabushi,คน,โชคดี,คน,รอบ,คิว,สาขา,...",0,0.997716
14,15,4621tm-shabushi-เดอะมอลล์บางกะปิ-ชั้น-g,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,7eb0c7d8683e407f83cc4d18e51d800d,ชาบู ชาบูชิ,สวัสดีครับวันนี้จะขอมารีวิวร้านชาบูชิ บุฟเฟขวั...,4.0,"สวัสดี,ชาบูชิ,บุฟเฟ,ขวัญใจ,คน,เมนู,ระบบ,สายพาน...",2,0.996361
15,16,4621tm-shabushi-เดอะมอลล์บางกะปิ-ชั้น-g,Shabushi (ชาบูชิ) เดอะมอลล์บางกะปิ ชั้น G,qvanqvan,บุฟเฟต์น้ำเดือด สายพานร่อน,ชาบูชิ สาขาเดอะมอลล์บางกะปิ ตั้งอยู่ประตูหน้าห...,4.0,"ชาบูชิ,สาขา,เดอะมอลล์,บางกะปิ,หน้า,ห้าง,แมคโดน...",2,0.998830


# **ข้าน้อยขอชาบู**

## Create Dictionary

In [ ]:
df_khanoy = df[df['Restaurant'] == 'ข้าน้อยขอชาบู']
df_khanoy.shape

(5, 8)

In [ ]:
documents_khanoy = df_khanoy['Review_tokenized'].to_list()
texts_khanoy = [[text for text in doc.split(',')] for doc in documents_khanoy]
dictionary_khanoy = gensim.corpora.Dictionary(texts_khanoy)

In [ ]:
print(dictionary_khanoy.token2id.keys())

dict_keys(['HALL', 'MCC', 'ของหวาน', 'คนละ', 'ค่ายมวย', 'ชั้น', 'ซุป', 'ซูชิ', 'ด้านใน', 'ตอนแรก', 'ตัวเลือก', 'ทอด', 'ที่ไหนได้', 'นารายพิซเซอเรีย', 'น้ำซุป', 'น้ำดื่ม', 'บางกะปิ', 'พิซซ่า', 'รสชาติ', 'รอบ', 'ราคา', 'รีฟีล', 'สาขา', 'หมู', 'หิว', 'อยากได้', 'เดอะมอลล์', 'เนื้อ', 'เบค่อน', 'เมนู', 'แกงเขียวหวาน', 'โซน', 'VAT', 'ชม', 'ชุด', 'ช่อง', 'ซีฟู้ด', 'ต้มยำ', 'ทอน', 'น้ำ', 'น้ำซึป', 'บุฟเฟ่ต์', 'ฟรี', 'ลอตเตอรี่', 'สัปดาห์', 'หม้อ', 'หัว', 'อันนี้', 'ฮันนี่โทส', 'เงิน', 'โปรโมชั่น', 'ได้เวลา', 'Narai pizzeria', 'mall', 'pizza', 'union', 'ขยัน', 'ของสด', 'ข้าวมันไก่', 'คุณภาพ', 'จำได้', 'ชื่อ', 'ตัดกำลัง', 'ตู้', 'นิวออลีน', 'บางส่วน', 'ปรุง', 'พนักงาน', 'สปาเกตตี้', 'สุก', 'อร่อย', 'ไก่', 'ไลน์', 'กก', 'คุ้มค่า', 'งาน', 'ท่าน', 'นุ่ม', 'บราวนี่', 'บุพเฟ่', 'บุฟเฟ่', 'ปิดท้าย', 'ผัก', 'รสชาต', 'วิปครีม', 'สมราคา', 'หอ', 'เซต', 'เด็ด', 'เรย', 'โชค', 'ไอติม', 'ตอน', 'สดๆ ร้อนๆ', 'หน้า', 'เงินสด', 'เนื้อวัว', 'แปลก', 'โอเค'])


In [ ]:
gensim_corpus_khanoy = [dictionary_khanoy.doc2bow(text, allow_update=True) for 
                        text in texts_khanoy]
word_frequencies_khanoy = [[(dictionary_khanoy[id], frequence) for 
                            id, frequence in couple] for 
                           couple in gensim_corpus_khanoy]

## Topic Modeling

In [ ]:
num_topics = 3
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary_khanoy[0]  # This is only to "load" the dictionary.
id2word_khanoy = dictionary_khanoy.id2token

%time model_khanoy = gensim.models.LdaModel(corpus=gensim_corpus_khanoy, 
                                            id2word=id2word_khanoy,
                                            chunksize=chunksize, \
                                            alpha='auto', eta='auto', \
                                            iterations=iterations, 
                                            num_topics=num_topics, \
                                            passes=passes, 
                                            eval_every=eval_every)

CPU times: user 85.8 ms, sys: 504 µs, total: 86.3 ms
Wall time: 88.8 ms


In [ ]:
pyLDAvis.gensim.prepare(model_khanoy, gensim_corpus_khanoy, dictionary_khanoy)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.031513 -0.010949       1        1  44.920896
1     -0.027397 -0.016758       2        1  34.049496
0     -0.004117  0.027707       3        1  21.029608, topic_info=        Term      Freq     Total Category  logprob  loglift
41  บุฟเฟ่ต์  2.000000  2.000000  Default  30.0000  30.0000
34       ชุด  1.000000  1.000000  Default  29.0000  29.0000
14    น้ำซุป  2.000000  2.000000  Default  28.0000  28.0000
39       น้ำ  2.000000  2.000000  Default  27.0000  27.0000
2    ของหวาน  4.000000  4.000000  Default  26.0000  26.0000
..       ...       ...       ...      ...      ...      ...
23       หมู  0.364083  2.221477   Topic3  -4.3784  -0.2493
71       ไก่  0.354068  2.180961   Topic3  -4.4063  -0.2588
20      ราคา  0.972696  6.572111   Topic3  -3.3957  -0.3513
2    ของหวาน  0.652482  4.402404   Topic3  -3.7950  -0.3499
70     อร่อย  0.638880  4.347032   Topic3  -3.8161  -0.3583

[158 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
0         1  0.878531            HALL
1         1  0.879526             MCC
52        1  0.916477  Narai pizzeria
32        1  0.490860             VAT
32        2  0.490860             VAT
...     ...       ...             ...
98        2  0.984848            โอเค
71        1  0.458513             ไก่
71        2  0.458513             ไก่
72        1  0.917658            ไลน์
91        2  0.983543           ไอติม

[108 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])

In [ ]:
model_khanoy.show_topics()

[(0,
  '0.046*"บุฟเฟ่ต์" + 0.037*"พิซซ่า" + 0.034*"ราคา" + 0.032*"น้ำซุป" + 0.025*"ชุด" + 0.024*"น้ำ" + 0.024*"คนละ" + 0.022*"ของหวาน" + 0.022*"อร่อย" + 0.020*"VAT"'),
 (1,
  '0.050*"ราคา" + 0.048*"ของหวาน" + 0.048*"อร่อย" + 0.039*"พิซซ่า" + 0.028*"น้ำ" + 0.021*"บุฟเฟ่" + 0.018*"ซูชิ" + 0.018*"หมู" + 0.018*"VAT" + 0.018*"โปรโมชั่น"'),
 (2,
  '0.052*"ราคา" + 0.035*"พิซซ่า" + 0.025*"คนละ" + 0.024*"ของหวาน" + 0.024*"อร่อย" + 0.020*"สาขา" + 0.020*"รสชาติ" + 0.020*"ชื่อ" + 0.020*"บางส่วน" + 0.019*"น้ำซุป"')]

In [ ]:
model_khanoy.show_topic(1)

[('ราคา', 0.050497673),
 ('ของหวาน', 0.047962107),
 ('อร่อย', 0.047555633),
 ('พิซซ่า', 0.039058734),
 ('น้ำ', 0.028193599),
 ('บุฟเฟ่', 0.021468159),
 ('ซูชิ', 0.018030768),
 ('หมู', 0.01802981),
 ('VAT', 0.017861737),
 ('โปรโมชั่น', 0.017839395)]

In [ ]:
df_khanoy['topics'] = df_khanoy['Review_tokenized'].apply(lambda x: model_khanoy.get_document_topics(dictionary_khanoy.doc2bow(x.split(',')))[0][0])
df_khanoy['score'] = df_khanoy['Review_tokenized'].apply(lambda x: model_khanoy.get_document_topics(dictionary_khanoy.doc2bow(x.split(',')))[0][1])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df_khanoy.head(10)

,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating,Review_tokenized,topics,score
16,17,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,ployynp,บุฟเฟ่ต์ชาบูและพิซซ่าไม่อั้นในราคา 199 บาท เน้...,หลังจากที่เคยลองสาขายูเนี่ยนมอลล์ไป รอบนี้มาที...,4.0,"สาขา,รอบ,สาขา,เดอะมอลล์,บางกะปิ,ชั้น,โซน,MCC,H...",2,0.998142
17,18,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,27a91236fe5e4559a4f097c97a480781,ร้านบุฟเฟ่ต์ ราคามิตรภาพ อยู่ชั้น4 ติดโรงหนัง ...,ร้านบุฟเฟ่ต์ที่มีโปรโมชั่นหัวละ199บาท ไม่รวมน้...,4.0,"บุฟเฟ่ต์,โปรโมชั่น,หัว,น้ำ,VAT,ได้เวลา,ชม,น้ำซ...",0,0.996984
18,19,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,0b81d251e4db486f9bcdba73b374ed99,ของหลากหลาย ปนๆ งงๆ นิดหน่อย,เคยรู้จักร้านนี้จากที่ union mall ไม่เคยได้ลอง...,3.0,"union,mall,จำได้,ชื่อ,ชื่อ,Narai pizzeria,ไลน์...",2,0.997063
19,20,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,40e0e087f3914fd49a8933b5a29936ca,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,อร่อยมากค่ะ คุ้มค่าสมราคา บุฟเฟ่หมูผักต่างๆ รว...,5.0,"อร่อย,คุ้มค่า,สมราคา,บุฟเฟ่,หมู,ผัก,น้ำ,ท่าน,เ...",1,0.997776
20,21,436045MJ-ข้าน้อยขอชาบู,ข้าน้อยขอชาบู,41841cb99ea243a3a8d4b006e946c586,แม้จะแปลกบ้าง แต่ก็ถือว่าอยู่ในเกณฑ์ที่ดี มีอา...,ก็ตามที่เขียนเลยครับ ว่า ถ้าจะจ่ายในราคา 199 บ...,NaN,"ราคา,เงินสด,ราคา,น้ำ,VAT,เนื้อวัว,ซูชิ,ของหวาน...",1,0.995275
